In [1]:
# Add your imports here
import pandas as pd
import numpy as np
import scipy as sp
from pyspark.sql import *
from pyspark.sql.types import *
import matplotlib.pyplot as plt
%matplotlib inline

from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext

import re
import datetime
import os
import math

# create the session
spark = SparkSession.builder.getOrCreate()

# create the context
sc = spark.sparkContext

In [2]:
DATA_FOLDER = 'Data'
REVIEW_FOLDER_0 = DATA_FOLDER + '/amazon_reviews_us_Books_v1_00.tsv.gz'
REVIEW_FOLDER_1 = DATA_FOLDER + '/amazon_reviews_us_Books_v1_01.tsv.gz'
REVIEW_FOLDER_2 = DATA_FOLDER + '/amazon_reviews_us_Books_v1_02.tsv.gz'
COVER_FOLDER = DATA_FOLDER + '/book32-listing.csv'

In [4]:
#REVIEW_FOLDER_TEST = 'Data/amazon_reviews_us_Books_v1_00.tsv.gz'
#df_reviews_0 = spark.read.format('csv').option('delimiter', '\t').option('header', 'true').load(REVIEW_FOLDER_TEST)
#df_reviews_0.write.parquet('reviews_us_0.parquet', mode='overwrite')


In [7]:
# Add your code and description here
if os.path.exists(DATA_FOLDER+'/Reviews_us.parquet'):
    #load the dataset into a Spark dataframe
    df_reviews = spark.read.parquet(DATA_FOLDER+'/Reviews_us.parquet')
    #show the content (2 first rows)
    df_reviews.show(2)
else:
    #load the dataset into a Spark dataframe
    df_reviews_0 = spark.read.format('csv').option('delimiter', '\t').option('header', 'true').load(REVIEW_FOLDER_0)
    df_reviews_1 = spark.read.format('csv').option('delimiter', '\t').option('header', 'true').load(REVIEW_FOLDER_1)
    df_reviews_2 = spark.read.format('csv').option('delimiter', '\t').option('header', 'true').load(REVIEW_FOLDER_2)
    df_reviews = df_reviews_0.union(df_reviews_1).union(df_reviews_2)
    #show the content (2 first rows)
    df_reviews.show(2)
    #save it in the Parquet format
    df_reviews.write.parquet(DATA_FOLDER+'/Reviews_us.parquet', mode='overwrite')

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   25933450| RJOVP071AVAJO|0439873800|      84656342|There Was an Old ...|           Books|          5|            0|          0|   N|                Y|          Five Stars|I love it and so ...| 2015-08-31|
|         US|    1801372|R1ORGBETCDW3AI|1623953553|     729938122|      I Saw a Friend|           Books|          5|    

In [3]:
# Add your code and description here
if os.path.exists(DATA_FOLDER+'/Cover.parquet'):
    print("Loading from existing parquet...")
    #load the dataset into a Spark dataframe
    df_covers = spark.read.parquet(DATA_FOLDER+'/Cover.parquet')
    #show the content (2 first rows)
    df_covers.show(2)
else:
    print("Loading from csv...")
    #load the dataset into a Spark dataframe
    schema = StructType([
        StructField('product_id', StringType(), True), 
        StructField('filename', StringType(), True),
        StructField('image_url', StringType(), True),
        StructField('title', StringType(), True),
        StructField('author', StringType(), True),
        StructField('category_id', StringType(), True),
        StructField('category', StringType(), True)])
    df_covers = spark.read.format('csv').schema(schema).load(COVER_FOLDER)
    #show the content (2 first rows)
    df_covers.show(2)
    #save it in the Parquet format
    df_covers.write.parquet(DATA_FOLDER+'/Cover.parquet', mode='overwrite')

Loading from existing parquet...
+----------+--------------+--------------------+--------------------+------------------+-----------+----------------+
|product_id|      filename|           image_url|               title|            author|category_id|        category|
+----------+--------------+--------------------+--------------------+------------------+-----------+----------------+
|1285859480|1285859480.jpg|http://ecx.images...|      Microeconomics|     William Boyes|          2|Business & Money|
| 133885070|0133885070.jpg|http://ecx.images...|Economics Today: ...|Roger LeRoy Miller|          2|Business & Money|
+----------+--------------+--------------------+--------------------+------------------+-----------+----------------+
only showing top 2 rows



In [11]:
df_reviews_2 = spark.read.parquet(DATA_FOLDER+'/reviews_us_2.parquet')

In [12]:
df_reviews_2.count()

3105520

In [6]:
rdd_test = df_reviews_2.rdd

In [10]:
rdd_test.toDF()

DataFrame[marketplace: string, customer_id: string, review_id: string, product_id: string, product_parent: string, product_title: string, product_category: string, star_rating: string, helpful_votes: string, total_votes: string, vine: string, verified_purchase: string, review_headline: string, review_body: string, review_date: string]

In [13]:
df_test = df_reviews_2.withColumn('index', monotonically_increasing_id())


In [14]:
df_test_1 = df_test.sort('index').limit(100)

In [15]:
df_test_1.show(5)

Py4JJavaError: An error occurred while calling o88.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 6 in stage 11.0 failed 1 times, most recent failure: Lost task 6.0 in stage 11.0 (TID 47, localhost, executor driver): java.lang.OutOfMemoryError: Java heap space
	at org.apache.parquet.hadoop.ParquetFileReader$ConsecutiveChunkList.readAll(ParquetFileReader.java:1064)
	at org.apache.parquet.hadoop.ParquetFileReader.readNextRowGroup(ParquetFileReader.java:698)
	at org.apache.spark.sql.execution.datasources.parquet.VectorizedParquetRecordReader.checkEndOfRowGroup(VectorizedParquetRecordReader.java:303)
	at org.apache.spark.sql.execution.datasources.parquet.VectorizedParquetRecordReader.nextBatch(VectorizedParquetRecordReader.java:258)
	at org.apache.spark.sql.execution.datasources.parquet.VectorizedParquetRecordReader.nextKeyValue(VectorizedParquetRecordReader.java:161)
	at org.apache.spark.sql.execution.datasources.RecordReaderIterator.hasNext(RecordRe

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 50294)
Traceback (most recent call last):
  File "/anaconda/lib/python3.6/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/anaconda/lib/python3.6/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/anaconda/lib/python3.6/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/anaconda/lib/python3.6/socketserver.py", line 696, in __init__
    self.handle()
  File "/opt/spark/python/pyspark/accumulators.py", line 265, in handle
    poll(accum_updates)
  File "/opt/spark/python/pyspark/accumulators.py", line 238, in poll
    if func():
  File "/opt/spark/python/pyspark/accumulators.py", line 242, in accum_updates
    num_updates = read_int(self.rfile)
  File "/opt/spark/python/pyspark/serializers.py", line 69

In [12]:
df_test = df_reviews_2.withColumn('index', monotonically_increasing_id())
# sort ascending and take first 100 rows for df1
df_test_1 = df_test.sort('index').limit(100)

# sort descending and take 400 rows for df2
#df_test_2 = df_test.sort('index', ascending=False).limit(400)

In [13]:
df_test_1.show(5)

--- Logging error ---
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:49987)
Traceback (most recent call last):
  File "/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2881, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-13-6d5c0f1ab7a1>", line 1, in <module>
    df_test_1.show(5)
  File "/opt/spark/python/pyspark/sql/dataframe.py", line 350, in show
    print(self._jdf.showString(n, 20, vertical))
  File "/anaconda/lib/python3.6/site-packages/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/opt/spark/python/pyspark/sql/utils.py", line 63, in deco
    return f(*a, **kw)
  File "/anaconda/lib/python3.6/site-packages/py4j/protocol.py", line 328, in get_return_value
    format(target_id, ".", name), value)
py4j.protocol.Py4JJavaError: <unprintable Py4JJavaError object>

During handling of the above excepti

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:49987)
Traceback (most recent call last):
  File "/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2881, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-13-6d5c0f1ab7a1>", line 1, in <module>
    df_test_1.show(5)
  File "/opt/spark/python/pyspark/sql/dataframe.py", line 350, in show
    print(self._jdf.showString(n, 20, vertical))
  File "/anaconda/lib/python3.6/site-packages/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/opt/spark/python/pyspark/sql/utils.py", line 63, in deco
    return f(*a, **kw)
  File "/anaconda/lib/python3.6/site-packages/py4j/protocol.py", line 328, in get_return_value
    format(target_id, ".", name), value)
py4j.protocol.Py4JJavaError: <unprintable Py4JJavaError object>

During handling of the above exception, another exception 

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:49987)
Traceback (most recent call last):
  File "/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2881, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-13-6d5c0f1ab7a1>", line 1, in <module>
    df_test_1.show(5)
  File "/opt/spark/python/pyspark/sql/dataframe.py", line 350, in show
    print(self._jdf.showString(n, 20, vertical))
  File "/anaconda/lib/python3.6/site-packages/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/opt/spark/python/pyspark/sql/utils.py", line 63, in deco
    return f(*a, **kw)
  File "/anaconda/lib/python3.6/site-packages/py4j/protocol.py", line 328, in get_return_value
    format(target_id, ".", name), value)
py4j.protocol.Py4JJavaError: <unprintable Py4JJavaError object>

During handling of the above exception, another exception 

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:49987)
Traceback (most recent call last):
  File "/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2881, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-13-6d5c0f1ab7a1>", line 1, in <module>
    df_test_1.show(5)
  File "/opt/spark/python/pyspark/sql/dataframe.py", line 350, in show
    print(self._jdf.showString(n, 20, vertical))
  File "/anaconda/lib/python3.6/site-packages/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/opt/spark/python/pyspark/sql/utils.py", line 63, in deco
    return f(*a, **kw)
  File "/anaconda/lib/python3.6/site-packages/py4j/protocol.py", line 328, in get_return_value
    format(target_id, ".", name), value)
py4j.protocol.Py4JJavaError: <unprintable Py4JJavaError object>

During handling of the above exception, another exception 

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:49987)
Traceback (most recent call last):
  File "/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2881, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-13-6d5c0f1ab7a1>", line 1, in <module>
    df_test_1.show(5)
  File "/opt/spark/python/pyspark/sql/dataframe.py", line 350, in show
    print(self._jdf.showString(n, 20, vertical))
  File "/anaconda/lib/python3.6/site-packages/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/opt/spark/python/pyspark/sql/utils.py", line 63, in deco
    return f(*a, **kw)
  File "/anaconda/lib/python3.6/site-packages/py4j/protocol.py", line 328, in get_return_value
    format(target_id, ".", name), value)
py4j.protocol.Py4JJavaError: <unprintable Py4JJavaError object>

During handling of the above exception, another exception 

...

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:49987)
Traceback (most recent call last):
  File "/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2881, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-13-6d5c0f1ab7a1>", line 1, in <module>
    df_test_1.show(5)
  File "/opt/spark/python/pyspark/sql/dataframe.py", line 350, in show
    print(self._jdf.showString(n, 20, vertical))
  File "/anaconda/lib/python3.6/site-packages/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/opt/spark/python/pyspark/sql/utils.py", line 63, in deco
    return f(*a, **kw)
  File "/anaconda/lib/python3.6/site-packages/py4j/protocol.py", line 328, in get_return_value
    format(target_id, ".", name), value)
py4j.protocol.Py4JJavaError: <unprintable Py4JJavaError object>

During handling of the above exception, another exception 

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:49987)
Traceback (most recent call last):
  File "/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2881, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-13-6d5c0f1ab7a1>", line 1, in <module>
    df_test_1.show(5)
  File "/opt/spark/python/pyspark/sql/dataframe.py", line 350, in show
    print(self._jdf.showString(n, 20, vertical))
  File "/anaconda/lib/python3.6/site-packages/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/opt/spark/python/pyspark/sql/utils.py", line 63, in deco
    return f(*a, **kw)
  File "/anaconda/lib/python3.6/site-packages/py4j/protocol.py", line 328, in get_return_value
    format(target_id, ".", name), value)
py4j.protocol.Py4JJavaError: <unprintable Py4JJavaError object>

During handling of the above exception, another exception 

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:49987)
Traceback (most recent call last):
  File "/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2881, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-13-6d5c0f1ab7a1>", line 1, in <module>
    df_test_1.show(5)
  File "/opt/spark/python/pyspark/sql/dataframe.py", line 350, in show
    print(self._jdf.showString(n, 20, vertical))
  File "/anaconda/lib/python3.6/site-packages/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/opt/spark/python/pyspark/sql/utils.py", line 63, in deco
    return f(*a, **kw)
  File "/anaconda/lib/python3.6/site-packages/py4j/protocol.py", line 328, in get_return_value
    format(target_id, ".", name), value)
py4j.protocol.Py4JJavaError: <unprintable Py4JJavaError object>

During handling of the above exception, another exception 

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:49987)
Traceback (most recent call last):
  File "/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2881, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-13-6d5c0f1ab7a1>", line 1, in <module>
    df_test_1.show(5)
  File "/opt/spark/python/pyspark/sql/dataframe.py", line 350, in show
    print(self._jdf.showString(n, 20, vertical))
  File "/anaconda/lib/python3.6/site-packages/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/opt/spark/python/pyspark/sql/utils.py", line 63, in deco
    return f(*a, **kw)
  File "/anaconda/lib/python3.6/site-packages/py4j/protocol.py", line 328, in get_return_value
    format(target_id, ".", name), value)
py4j.protocol.Py4JJavaError: <unprintable Py4JJavaError object>

During handling of the above exception, another exception 

Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:49987)

In [8]:
df_joined = df_reviews_2.join(df_covers, ['product_id'])

In [9]:
df_joined.show(2)

--- Logging error ---
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:49914)
Traceback (most recent call last):
  File "/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2881, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-9-9bd12c25d916>", line 1, in <module>
    df_joined.show(2)
  File "/opt/spark/python/pyspark/sql/dataframe.py", line 350, in show
    print(self._jdf.showString(n, 20, vertical))
  File "/anaconda/lib/python3.6/site-packages/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/opt/spark/python/pyspark/sql/utils.py", line 63, in deco
    return f(*a, **kw)
  File "/anaconda/lib/python3.6/site-packages/py4j/protocol.py", line 328, in get_return_value
    format(target_id, ".", name), value)
py4j.protocol.Py4JJavaError: <unprintable Py4JJavaError object>

During handling of the above exceptio

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:49914)
Traceback (most recent call last):
  File "/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2881, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-9-9bd12c25d916>", line 1, in <module>
    df_joined.show(2)
  File "/opt/spark/python/pyspark/sql/dataframe.py", line 350, in show
    print(self._jdf.showString(n, 20, vertical))
  File "/anaconda/lib/python3.6/site-packages/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/opt/spark/python/pyspark/sql/utils.py", line 63, in deco
    return f(*a, **kw)
  File "/anaconda/lib/python3.6/site-packages/py4j/protocol.py", line 328, in get_return_value
    format(target_id, ".", name), value)
py4j.protocol.Py4JJavaError: <unprintable Py4JJavaError object>

During handling of the above exception, another exception o

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:49914)
Traceback (most recent call last):
  File "/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2881, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-9-9bd12c25d916>", line 1, in <module>
    df_joined.show(2)
  File "/opt/spark/python/pyspark/sql/dataframe.py", line 350, in show
    print(self._jdf.showString(n, 20, vertical))
  File "/anaconda/lib/python3.6/site-packages/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/opt/spark/python/pyspark/sql/utils.py", line 63, in deco
    return f(*a, **kw)
  File "/anaconda/lib/python3.6/site-packages/py4j/protocol.py", line 328, in get_return_value
    format(target_id, ".", name), value)
py4j.protocol.Py4JJavaError: <unprintable Py4JJavaError object>

During handling of the above exception, another exception o

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:49914)
Traceback (most recent call last):
  File "/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2881, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-9-9bd12c25d916>", line 1, in <module>
    df_joined.show(2)
  File "/opt/spark/python/pyspark/sql/dataframe.py", line 350, in show
    print(self._jdf.showString(n, 20, vertical))
  File "/anaconda/lib/python3.6/site-packages/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/opt/spark/python/pyspark/sql/utils.py", line 63, in deco
    return f(*a, **kw)
  File "/anaconda/lib/python3.6/site-packages/py4j/protocol.py", line 328, in get_return_value
    format(target_id, ".", name), value)
py4j.protocol.Py4JJavaError: <unprintable Py4JJavaError object>

During handling of the above exception, another exception o

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:49914)
Traceback (most recent call last):
  File "/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2881, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-9-9bd12c25d916>", line 1, in <module>
    df_joined.show(2)
  File "/opt/spark/python/pyspark/sql/dataframe.py", line 350, in show
    print(self._jdf.showString(n, 20, vertical))
  File "/anaconda/lib/python3.6/site-packages/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/opt/spark/python/pyspark/sql/utils.py", line 63, in deco
    return f(*a, **kw)
  File "/anaconda/lib/python3.6/site-packages/py4j/protocol.py", line 328, in get_return_value
    format(target_id, ".", name), value)
py4j.protocol.Py4JJavaError: <unprintable Py4JJavaError object>

During handling of the above exception, another exception o

...

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:49914)
Traceback (most recent call last):
  File "/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2881, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-9-9bd12c25d916>", line 1, in <module>
    df_joined.show(2)
  File "/opt/spark/python/pyspark/sql/dataframe.py", line 350, in show
    print(self._jdf.showString(n, 20, vertical))
  File "/anaconda/lib/python3.6/site-packages/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/opt/spark/python/pyspark/sql/utils.py", line 63, in deco
    return f(*a, **kw)
  File "/anaconda/lib/python3.6/site-packages/py4j/protocol.py", line 328, in get_return_value
    format(target_id, ".", name), value)
py4j.protocol.Py4JJavaError: <unprintable Py4JJavaError object>

During handling of the above exception, another exception o

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:49914)
Traceback (most recent call last):
  File "/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2881, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-9-9bd12c25d916>", line 1, in <module>
    df_joined.show(2)
  File "/opt/spark/python/pyspark/sql/dataframe.py", line 350, in show
    print(self._jdf.showString(n, 20, vertical))
  File "/anaconda/lib/python3.6/site-packages/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/opt/spark/python/pyspark/sql/utils.py", line 63, in deco
    return f(*a, **kw)
  File "/anaconda/lib/python3.6/site-packages/py4j/protocol.py", line 328, in get_return_value
    format(target_id, ".", name), value)
py4j.protocol.Py4JJavaError: <unprintable Py4JJavaError object>

During handling of the above exception, another exception o

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:49914)
Traceback (most recent call last):
  File "/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2881, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-9-9bd12c25d916>", line 1, in <module>
    df_joined.show(2)
  File "/opt/spark/python/pyspark/sql/dataframe.py", line 350, in show
    print(self._jdf.showString(n, 20, vertical))
  File "/anaconda/lib/python3.6/site-packages/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/opt/spark/python/pyspark/sql/utils.py", line 63, in deco
    return f(*a, **kw)
  File "/anaconda/lib/python3.6/site-packages/py4j/protocol.py", line 328, in get_return_value
    format(target_id, ".", name), value)
py4j.protocol.Py4JJavaError: <unprintable Py4JJavaError object>

During handling of the above exception, another exception o

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:49914)
Traceback (most recent call last):
  File "/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2881, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-9-9bd12c25d916>", line 1, in <module>
    df_joined.show(2)
  File "/opt/spark/python/pyspark/sql/dataframe.py", line 350, in show
    print(self._jdf.showString(n, 20, vertical))
  File "/anaconda/lib/python3.6/site-packages/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/opt/spark/python/pyspark/sql/utils.py", line 63, in deco
    return f(*a, **kw)
  File "/anaconda/lib/python3.6/site-packages/py4j/protocol.py", line 328, in get_return_value
    format(target_id, ".", name), value)
py4j.protocol.Py4JJavaError: <unprintable Py4JJavaError object>

During handling of the above exception, another exception o

Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:49914)

In [10]:
print(df_reviews.count(), df_joined.count())

10319090 1423654


We read the dataset containing the reviews. We have to specify the columns names because some rows have a higher count of cell than others. We want to see what is contained in those rows. 

In [2]:
cols = ["marketplace",
        "customer_id",
        "review_id",
        "product_id",
        "product_parent",
        "product_title",
        "product_category",
        "star_rating",
        "helpful_votes",
        "total_votes",
        "vine",
        "verified_purchase",
        "review_headline",
        "review_body",
        "review_date", 
        "undefined_1",
        "undefined_2",
        "undefined_3",
        "undefined_4",
        "undefined_5",
        "undefined_6",
        "undefined_7"
       ]

In [3]:
#df_reviews = pd.read_table("amazon_reviews_us_Books_v1_00.tsv", error_bad_lines=False)
df_reviews = pd.read_csv(REVIEW_FOLDER, names=cols, sep='\t')

NameError: name 'pd' is not defined

The warning simply indicates that some columns do not have the same dtype for each row, this will be handled later by cleaning the data. 

So first we check the rows with a higher count of cells.

In [92]:
df_reviews[df_reviews["undefined_1"].notnull()]

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,...,review_headline,review_body,review_date,undefined_1,undefined_2,undefined_3,undefined_4,undefined_5,undefined_6,undefined_7
3523,US,45223684,R1NHKVD5LMP2JY,1503551814,353548609,A Nutty World: On the Edge of the Rain Forest,Books,5,0,0,...,She is a fine storyteller and has a message c...,Books,5,0.0,0.0,N,Y,looks great.,Very durable. looks great.,2015-08-31
5281,US,18701079,R1946J1OJKC1K2,0743292472,7685359,The Ultimate Battle: Okinawa 1945--The Last Ep...,Books,5,0,0,...,was a great read. Quick-moving with sharp int...,Books,1,6.0,7.0,N,Y,NOT PLEASED WITH THE VENDORS AT ALL,THE CASE IS MADE OF A CHEAP QUALITY. IT BROKE ...,2015-08-31
20477,US,40945620,R33MH6R1TZWIGR,1452572747,523136939,"Have This Mind: Supreme Happiness, Ultimate Re...",Books,5,1,1,...,This wonderful book is thoughtful and well-wri...,Books,5,0.0,0.0,N,Y,Very good!,Everything as promised. Very good!,2015-08-30
25894,US,22921197,R1OPQAX1D7DWB8,0984755101,632337521,The Primal Connection: Follow Your Genetic Blu...,Books,5,0,0,...,this is the best book on health and wellness ...,Books,4,1.0,2.0,N,N,Don't Be Mislead...,This is not so much intended to be a review as...,2015-08-29
27015,US,44087253,R24H2K78E883A0,1616496169,709191612,Navigating Grace: A Solo Voyage of Survival an...,Books,5,2,2,...,"Oh would some Power the gift give us, To ...\t...",Books,5,0.0,0.0,N,Y,It was a great experience and one I will alway...,After reading the book I visited Helen and too...,2015-08-29
59797,US,44250036,RYNEKW5P1Z3AG,0385262078,887509044,The Impossible H L. Mencken,Books,5,0,0,...,One of the most important of all American men...,Books,5,0.0,0.0,N,Y,The storyline is great and the characters are ...,This book is the third in the series and in my...,2015-08-26
69197,US,50641462,R2VI5KEAQR2P23,0545436826,247155770,Friendshape,Books,3,6,6,...,It is a bunch of sweet musings about friendsh...,Books,5,0.0,0.0,N,N,... have heard a few things about Medjugorje b...,I have heard a few things about Medjugorje but...,2015-08-25
71952,US,43676679,R2F0SUDYHR79JQ,1844094642,766424526,"Change Your Story, Change Your Life: Using Sha...",Books,2,4,16,...,I intensely dislike the psychology perspectiv...,Books,5,1.0,2.0,N,Y,Five Stars,love it,2015-08-25
78719,US,4881222,R21NUSCF1WC1I8,1607512572,236705009,The House on Tradd Street,Books,4,0,0,...,discriptive of characters w/repetitive plot di...,Books,5,7.0,8.0,N,N,I like Kevin Trudeau - great information yet ...,I like Kevin Trudeau because he has made it hi...,2015-08-24
81299,US,1148395,R1UH3R71GX97AR,1502716984,563128335,Jack The Ripper: Newly Discovered Adventures o...,Books,1,1,2,...,is a complete waste of time and money\tthis &...,Books,5,0.0,0.0,N,Y,Five Stars,This item was received. Thank you.,2015-08-24


In [93]:
len(df_reviews[df_reviews["undefined_1"].notnull()])

326

What is contained in the undefined columns is a bit weird, it's like we have more reviews, but they are not really related to the standard review column. For example, in the second line of the output, the review is "Was a great read..." and then we have "NOT PLEASED WITH THE VENDORS AT ALL" etc.

Since these lines are not clear concerning the reviews, and since we only have 326 of such rows over 6 millions, we decide to drop these rows. 

In [ ]:
df_reviews = df_reviews[df_reviews[df_reviews["undefined_1"].isnull()]]

In [91]:
df_reviews[df_reviews["helpful_votes"].astype('str').apply(
    lambda x: int(x) if x.isnumeric() else np.nan) > 100]

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,...,review_headline,review_body,review_date,undefined_1,undefined_2,undefined_3,undefined_4,undefined_5,undefined_6,undefined_7
2093,US,15365576,R38I0D0J8O6LB6,048680318X,338332455,Creative Haven Midnight Garden Coloring Book: ...,Books,5,101,102,...,"Flowers, Leaves, hearts, & vines combined and ...",This book is filled with 31 artistic and imagi...,2015-08-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7914,US,43484359,R2T373PSIAQURD,162315569X,635788774,Modern Dutch Oven Cookbook: Fresh Ideas for Br...,Books,2,147,152,...,Not what I was hoping for,I bought this based on the rave reviews but it...,2015-08-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9107,US,30673354,R1F7PAD7F00VUI,1119057027,373857330,Teach Yourself VISUALLY Windows 10 (Teach Your...,Books,3,123,125,...,"Good for quick, basic navigation. Limited.",I am using Windows 10 and own a copy of this b...,2015-08-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11505,US,50489373,R1HO6TV1G31KA9,1623363586,465642569,Thug Kitchen: The Official Cookbook: Eat Like ...,Books,1,103,143,...,Not very helpful. Profanity doesn't substitute...,It kinda read like this to me:<br />Hey you la...,2015-08-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11998,US,13098534,RE2KQV5I1AS5Z,1780287518,268093982,Chakra Wisdom Oracle Cards: The Complete Spiri...,Books,1,300,324,...,Lame,I wanted an oracle deck that was related to ch...,2015-08-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16284,US,52751117,R3E34ZT1227WIF,1477829407,529331494,The Einstein Prophecy,Books,1,323,371,...,Since the author didn't bother to verify even ...,If you are going to locate a novel in a recent...,2015-08-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19789,US,45943820,R2JAD68WXL1PSY,0007525540,669379389,The Lord of the Rings,Books,5,192,213,...,A great edition for the price,Review for [[ASIN:0007525540 The Lord of the R...,2015-08-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21361,US,52936144,R34VXM3EJDS78Z,1594634718,892490739,Big Magic: Creative Living Beyond Fear,Books,3,374,410,...,One creative's engaging and personal take on h...,"Given the success of her earlier works, it's p...",2015-08-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34930,US,13768935,R2JF5N8P1JTJVE,0486799026,424027076,Creative Haven Dream Doodles: A Coloring Book ...,Books,5,111,116,...,I absolutely love the details Kathy puts into ...,Just finished my first page of the book. I ab...,2015-08-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35222,US,29532845,R3SVHW4TTDH9KF,0812995821,604086967,Rising Strong: The Reckoning. The Rumble. The ...,Books,3,331,358,...,"I'm a Fan, But This Book Missed the Mark",I have loved Brene Brown's books but this one ...,2015-08-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [73]:
df_reviews[df_reviews["undefined_1"].notnull()].iloc[:, 10:]

,vine,verified_purchase,review_headline,review_body,review_date,undefined_1,undefined_2,undefined_3,undefined_4,undefined_5,undefined_6,undefined_7
3523,N,Y,She is a fine storyteller and has a message c...,Books,5,0.0,0.0,N,Y,looks great.,Very durable. looks great.,2015-08-31
5281,N,Y,was a great read. Quick-moving with sharp int...,Books,1,6.0,7.0,N,Y,NOT PLEASED WITH THE VENDORS AT ALL,THE CASE IS MADE OF A CHEAP QUALITY. IT BROKE ...,2015-08-31
20477,N,Y,This wonderful book is thoughtful and well-wri...,Books,5,0.0,0.0,N,Y,Very good!,Everything as promised. Very good!,2015-08-30
25894,N,Y,this is the best book on health and wellness ...,Books,4,1.0,2.0,N,N,Don't Be Mislead...,This is not so much intended to be a review as...,2015-08-29
27015,N,Y,"Oh would some Power the gift give us, To ...\t...",Books,5,0.0,0.0,N,Y,It was a great experience and one I will alway...,After reading the book I visited Helen and too...,2015-08-29
59797,N,Y,One of the most important of all American men...,Books,5,0.0,0.0,N,Y,The storyline is great and the characters are ...,This book is the third in the series and in my...,2015-08-26
69197,N,Y,It is a bunch of sweet musings about friendsh...,Books,5,0.0,0.0,N,N,... have heard a few things about Medjugorje b...,I have heard a few things about Medjugorje but...,2015-08-25
71952,N,Y,I intensely dislike the psychology perspectiv...,Books,5,1.0,2.0,N,Y,Five Stars,love it,2015-08-25
78719,N,Y,discriptive of characters w/repetitive plot di...,Books,5,7.0,8.0,N,N,I like Kevin Trudeau - great information yet ...,I like Kevin Trudeau because he has made it hi...,2015-08-24
81299,N,Y,is a complete waste of time and money\tthis &...,Books,5,0.0,0.0,N,Y,Five Stars,This item was received. Thank you.,2015-08-24


In [53]:
test3[test3["product_id"] == "014006768X"].head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
33670,US,31269475,R2VT0IUKEZ1IZI,014006768X,98817612,The First Rumpole Omnibus,Books,5,0.0,1.0,N,Y,Great fun!,"Mortimer and his brain-child, Rumpole are both...",2015-08-28
566408,US,10029757,RK2X98AB8WAVH,014006768X,98817612,The First Rumpole Omnibus,Books,5,0.0,1.0,N,Y,Entertaing,I love it the only two author's that could bri...,2015-07-10
776844,US,16010139,R3SI9L25CWYZ4I,014006768X,98817612,The First Rumpole Omnibus,Books,5,1.0,1.0,N,Y,Fortunately many of these stories were made in...,Fortunately many of these stories were made in...,2015-06-19
885302,US,16587837,R3TALFOQHVFEFN,014006768X,98817612,The First Rumpole Omnibus,Books,5,1.0,1.0,N,N,Humanity on parade,I was sitting alone at the bar of a coastal sa...,2015-06-09
1791515,US,33892769,R19U49VOZRB3ZN,014006768X,98817612,The First Rumpole Omnibus,Books,5,0.0,1.0,N,Y,Five Stars,Can't beat these books1,2015-03-16


In [25]:
test3.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,25933450,RJOVP071AVAJO,0439873800,84656342,There Was an Old Lady Who Swallowed a Shell!,Books,5,0.0,0.0,N,Y,Five Stars,I love it and so does my students!,2015-08-31
1,US,1801372,R1ORGBETCDW3AI,1623953553,729938122,I Saw a Friend,Books,5,0.0,0.0,N,Y,"Please buy ""I Saw a Friend""! Your children wil...",My wife and I ordered 2 books and gave them as...,2015-08-31
2,US,5782091,R7TNRFQAOUTX5,142151981X,678139048,"Black Lagoon, Vol. 6",Books,5,0.0,0.0,N,Y,Shipped fast.,Great book just like all the others in the ser...,2015-08-31
3,US,32715830,R2GANXKDIFZ6OI,014241543X,712432151,If I Stay,Books,5,0.0,0.0,N,N,Five Stars,So beautiful,2015-08-31
4,US,14005703,R2NYB6C3R8LVN6,1604600527,800572372,Stars 'N Strips Forever,Books,5,2.0,2.0,N,Y,Five Stars,Enjoyed the author's story and his quilts are ...,2015-08-31


In [58]:
test = test3["star_rating"].astype('str').apply(lambda x: x if x.isnumeric() else np.nan).dropna()


0           5
1           5
2           5
3           5
4           5
5           2
6           5
7           5
8           5
9           5
10          5
11          5
12          5
13          4
14          3
15          5
16          5
17          5
18          5
19          5
20          5
21          5
22          5
23          5
24          5
25          5
26          5
27          5
28          3
29          5
           ..
10236820    5
10236821    5
10236822    5
10236823    5
10236824    5
10236825    5
10236826    3
10236827    5
10236828    5
10236829    4
10236830    5
10236831    4
10236832    5
10236833    5
10236834    5
10236835    5
10236836    5
10236837    4
10236838    5
10236839    3
10236840    5
10236841    1
10236842    3
10236843    4
10236844    5
10236845    5
10236846    4
10236847    4
10236848    5
10236849    5
Name: star_rating, Length: 8205186, dtype: object

In [ ]:
test3[test3["star_rating"] == "2015-08-04"]

In [54]:
test3[test3["star_rating"] == "2015-08-04"]

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
299292,US,42182727,R3IUDW7J4XLFHV,B003V8JQAI,198998645,Dare to Be Great Ms. Caucus a Doonebury Book\t...,"This is a very objective, highly interesting, ...",2015-08-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN
303458,US,52159729,R2S7P4O3PDJKSL,0988710501,101892350,"Prostate Cancer Breakthroughs 2014: New Tests,...",Harry Turtledove is an award-winning writer of...,2015-08-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
test3.loc[:, "star_rating"].value_counts()

5             5829705
5             1365286
4             1315264
3              567500
1              344802
4              282042
2              278990
3              121547
1               73240
2               58381
2015-08-04          2
2015-03-27          2
2015-06-11          1
2015-02-25          1
2014-07-24          1
2015-05-02          1
2015-07-06          1
2014-03-04          1
2015-05-27          1
2013-04-17          1
2015-04-05          1
2014-11-24          1
2013-05-28          1
2014-10-13          1
2015-06-05          1
2015-07-12          1
2015-05-20          1
2015-08-25          1
2014-12-01          1
2014-11-12          1
2014-09-29          1
2015-06-14          1
2015-04-17          1
2014-08-25          1
2013-08-24          1
2015-08-08          1
2015-05-29          1
2015-08-31          1
2013-08-03          1
2015-07-15          1
2015-05-24          1
Name: star_rating, dtype: int64

In [17]:
ratings = test3.loc[:, "star_rating"]

In [23]:
(ratings == 5).sum()

5829705

In [24]:
(ratings == "5").sum()

1365286

In [31]:
covers = pd.read_csv("book32-listing.csv", encoding = "ISO-8859-1", header=None)

In [32]:
columns = ["[AMAZON INDEX (ASIN)}","[FILENAME]","[IMAGE URL]","[TITLE]","[AUTHOR]","[CATEGORY ID]","[CATEGORY]"]

In [34]:
covers.columns = columns

In [37]:
covers.loc[10010:10020, :]

,[AMAZON INDEX (ASIN)},[FILENAME],[IMAGE URL],[TITLE],[AUTHOR],[CATEGORY ID],[CATEGORY]
10010,385353596,0385353596.jpg,http://ecx.images-amazon.com/images/I/51iNYVpL...,The Jezebel Remedy: A novel,Martin Clark,17,"Mystery, Thriller & Suspense"
10011,014006768X,014006768X.jpg,http://ecx.images-amazon.com/images/I/51pTgRqM...,The First Rumpole Omnibus,John Mortimer,17,"Mystery, Thriller & Suspense"
10012,098544648X,098544648X.jpg,http://ecx.images-amazon.com/images/I/51CYTK4O...,Cage Without Bars,Gerald Neufeld,17,"Mystery, Thriller & Suspense"
10013,440243823,0440243823.jpg,http://ecx.images-amazon.com/images/I/51vgdKFN...,The Associate: A Novel,John Grisham,17,"Mystery, Thriller & Suspense"
10014,312033567,0312033567.jpg,http://ecx.images-amazon.com/images/I/41G8381G...,Anatomy of a Murder,Robert Traver,17,"Mystery, Thriller & Suspense"
10015,038550120X,038550120X.jpg,http://ecx.images-amazon.com/images/I/51NAC46Q...,A Painted House,John Grisham,17,"Mystery, Thriller & Suspense"
10016,385339666,0385339666.jpg,http://ecx.images-amazon.com/images/I/519R15FS...,The Chamber,John Grisham,17,"Mystery, Thriller & Suspense"
10017,1594136637,1594136637.jpg,http://ecx.images-amazon.com/images/I/51LWl64G...,Tuesdays Child (Thorndike Press Large Print Core),Fern Michaels,17,"Mystery, Thriller & Suspense"
10018,B00CL62PS6,B00CL62PS6.jpg,http://ecx.images-amazon.com/images/I/51vdTEk4...,"An Innocent Client: Joe Dillard, Book 1",Scott Pratt,17,"Mystery, Thriller & Suspense"
10019,316203440,0316203440.jpg,http://ecx.images-amazon.com/images/I/516H8c-H...,"The Lincoln Lawyer Novels: The Lincoln Lawyer,...",Michael Connelly,17,"Mystery, Thriller & Suspense"
